In [ ]:
%cd content/drive/MyDrive/SpectrometerLearner2/Zarya/
!ls

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-137-b7296996c5fa>", line 1, in <cell line: 1>
    get_ipython().run_line_magic('cd', '..')
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 

In [ ]:
from csv_processor import CSVProcessor
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Activation

In [109]:
processor = CSVProcessor("../Training Data")
X_train, y_train = processor.split()
processor = CSVProcessor("../Testing Data")
X_test, y_test = processor.split()

In [112]:
def apply_data_augmentation(X_train, y_train, num_augmented_samples=1000):
    augmented_data = []
    augmented_labels = []

    for i in range(num_augmented_samples):
        # Randomly select an index from the original data
        index = np.random.randint(len(X_train))
        original_sample = X_train.iloc[index]
        label = y_train[index]

        # Apply data augmentation (you can customize these operations)
        augmented_sample = original_sample + np.random.normal(0, 0.1, original_sample.shape)  # Adding random noise
        augmented_sample *= np.random.uniform(0.9, 1.1)  # Scaling by a random factor
        if np.random.rand() < 0.5:
            augmented_sample = np.flip(augmented_sample)  # Randomly flipping the sample

        augmented_data.append(augmented_sample)
        augmented_labels.append(label)

    # Convert the augmented data and labels to numpy arrays
    augmented_data = np.array(augmented_data)
    augmented_labels = np.array(augmented_labels)

    # Concatenate the original data with the augmented data
    X_train_augmented = np.concatenate((X_train.values, augmented_data), axis=0)
    y_train_augmented = np.concatenate((y_train, augmented_labels), axis=0)

    return X_train_augmented, y_train_augmented

# Assuming X_train and y_train are your original data in pandas DataFrames
X_train_augmented, y_train_augmented = apply_data_augmentation(X_train.reset_index(drop=True), y_train, num_augmented_samples=1000)

In [116]:
X_train, y_train = X_train_augmented, y_train_augmented

In [117]:
scaler = StandardScaler()

# Fit the scaler on the training features and transform them
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test features using the fitted scaler
X_test_scaled = scaler.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [118]:
# 4. Split the Data into Training and Test Sets
X_train, X_test = X_train_scaled, X_test_scaled

In [ ]:
"""
def create_model(input_shape, num_classes, hidden_layer_sizes=(128,), activation='relu', dropout_rate=0.5, learning_rate=0.001):
    model = Sequential()
    model.add(Dense(hidden_layer_sizes[0], input_shape=(input_shape,), activation=None))
    model.add(BatchNormalization())
    model.add(Activation(activation))
    model.add(Dropout(dropout_rate))

    if len(hidden_layer_sizes) > 1:
        for units in hidden_layer_sizes[1:]:
            model.add(Dense(units, activation=None))
            model.add(BatchNormalization())
            model.add(Activation(activation))
            model.add(Dropout(dropout_rate))

    # Output layer with proper activation based on the number of classes
    if num_classes == 1:
        model.add(Dense(1, activation='sigmoid'))
        loss_function = 'binary_crossentropy'
    else:
        model.add(Dense(num_classes, activation='softmax'))
        loss_function = 'sparse_categorical_crossentropy'

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

    return model  # Return the model.

# Create a KerasClassifier with fixed epochs
model = KerasClassifier(build_fn=create_model, input_shape=input_shape, num_classes=num_classes, epochs=20, batch_size=32, verbose=0)

# Define the hyperparameters and their possible values
param_grid = {
    'hidden_layer_sizes': [(64,), (128,), (256,), (64, 64), (128, 128), (256, 256)],
    'activation': ['relu', 'tanh', 'sigmoid', 'leaky_relu'],
    'dropout_rate': [0.2, 0.3, 0.4],
    'learning_rate': [0.001, 0.01, 0.1],
}

# Results list to store each iteration's data
results = []

# Perform GridSearchCV with error_score='raise'
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, error_score='raise')
try:
    grid_result = grid.fit(X_train, y_train)

    # Get the best model and its accuracy
    best_model = grid_result.best_estimator_
    best_accuracy = grid_result.best_score_
    best_params = grid_result.best_params_

    # Store results in the list
    results.append({
        'Hidden_Layer_Sizes': best_params['hidden_layer_sizes'],
        'Activation': best_params['activation'],
        'Dropout_Rate': best_params['dropout_rate'],
        'Learning_Rate': best_params['learning_rate'],
        'Training_Accuracy': best_accuracy,
        'Testing_Accuracy': best_model.score(X_test, y_test)
    })

    # Convert the list to a DataFrame
    results_df = pd.DataFrame(results)

    # Save the DataFrame to a CSV file
    results_df.to_csv('hyperparameter_results.csv', index=False)

except Exception as e:
    print("Error occurred during GridSearchCV:", str(e))
"""

<ipython-input-65-09fc12ef6942>:29: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, input_shape=input_shape, num_classes=num_classes, epochs=20, batch_size=32, verbose=0)


In [131]:
input_shape = X_train.shape[1]
num_classes = len(np.unique(y_train))

model = Sequential([
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])




In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model to the training data
model.fit(X_train, y_train, batch_size=8, epochs=100, verbose=1)

In [ ]:
# Evaluate the model on the testing data
loss, accuracy = model.evaluate(X_test, y_test)

print("Testing Accuracy:", accuracy)

In [136]:
!git status

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
fatal: Unable to read current working directory: Transport endpoint is not connected
